In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


In [3]:
from tensorflow.keras.models import load_model
class_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'nt','tn']
from PIL import Image
import matplotlib.pyplot as plt
model = load_model('/content/OCR_cnn.h5')


In [2]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.0/899.0 kB 23.6 MB/s eta 0:00:00


In [13]:
from ultralytics import YOLO

modelplate = YOLO('/content/best (1).pt')
modelcar = YOLO('/content/yolo11n.pt')
modelsegm= YOLO('/content/segmentation.pt')
modelceint= YOLO('/content/best.pt')

In [5]:
import cv2

def convert_to_black_and_white(image):

    if image is None:
        print("Error: Image is empty or invalid.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    _, black_and_white = cv2.threshold(gray, 102, 255, cv2.THRESH_BINARY)
    return black_and_white


In [20]:
import cv2
import os
import numpy as np

class_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'NT', 'Tunis']

cap = cv2.VideoCapture("/content/vdc.mp4")
video_path_out = "output_video3.mp4"

ret, frame = cap.read()
H, W, _ = frame.shape
out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*"DIVX"), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

frame_count = 0  
crop_count = 0   


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    results = modelcar(frame)  

    
    filtered_boxes = []
    filtered_persons = []
    for box in results[0].boxes:
        if box.cls == 2 or box.cls == 5:  
            filtered_boxes.append(box)
        elif box.cls == 0:
            filtered_persons.append(box)

    
    annotated_frame = frame.copy()  
    for box in filtered_persons:
      x1, y1, x2, y2 = map(int, box.xyxy[0])
      color = (250, 250, 0)
      cv2.rectangle(annotated_frame, (x1, y1), (x1, y1), color, 2)
      cropped_person = frame[y1:y2, x1:x2]
      result_ceint_detected = modelceint.predict(cropped_person)
      for box in result_ceint_detected[0].boxes:
          cls = int(box.cls)
          if cls == 0:
            x111, y111, x222, y222 = map(int, box.xyxy[0])
            color = (125, 125, 125)
            cv2.rectangle(annotated_frame, (x1 + x111, y1 + y111), (x1 + x222, y1 + y222), color, 2)
            cv2.putText(annotated_frame, "Ceinture", (x1+x111, y1+y111 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    for box in filtered_boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0]) 
        confidence = box.conf[0]  
        class_id = int(box.cls[0])  

        color = (0, 255, 0) if class_id == 2 else (0, 0, 255)  
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)

        label = f"{'Car' if class_id == 2 else 'Bus'}: {confidence:.2f}"
        cv2.putText(annotated_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


        
        cropped_object = frame[y1:y2, x1:x2]
       
        
        result_pl_detected = modelplate.predict(cropped_object)  

       
        for box in result_pl_detected[0].boxes:  
            cls = int(box.cls)  
            if cls == 0:  
                x11, y11, x22, y22 = map(int, box.xyxy[0])  

                
                color = (255, 0, 0)  
                cv2.rectangle(annotated_frame, (x1 + x11, y1 + y11), (x1 + x22, y1 + y22), color, 2)

                
                plate = cropped_object[y11:y22, x11:x22]

                result_characters = modelsegm.predict(plate)  

                predictList = []

                
                for char_box in result_characters[0].boxes:
                  char_x1, char_y1, char_x2, char_y2 = map(int, char_box.xyxy[0])  

                  
                  character_img = plate[char_y1:char_y2, char_x1:char_x2]

                  character_img = cv2.resize(character_img, (28, 28))  

                  
                  character_img = cv2.cvtColor(character_img, cv2.COLOR_BGR2GRAY)

                  character_img = character_img.reshape(1, 28, 28, 1)  
                  character_img = character_img / 255.0  

                  
                  predictedOCR = model.predict(character_img)
                  predicted_class = np.argmax(predictedOCR, axis=1)[0]

                  predictList.append((char_x1, class_labels[predicted_class]))

                predictList.sort(key=lambda x: x[0])
                plate_text = ''.join([char[1] for char in predictList])

                cv2.putText(annotated_frame, plate_text, (x1 + x11, y1 + y11 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    out.write(annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 1 person, 1 car, 15.4ms
Speed: 3.9ms preprocess, 15.4ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 448x640 1 seatbelt, 40.7ms
Speed: 2.3ms preprocess, 40.7ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)

0: 416x640 (no detections), 27.5ms
Speed: 3.1ms preprocess, 27.5ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)

0: 384x640 1 person, 1 car, 9.6ms
Speed: 3.3ms preprocess, 9.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 448x640 1 seatbelt, 12.9ms
Speed: 2.1ms preprocess, 12.9ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)

0: 416x640 (no detections), 26.6ms
Speed: 2.1ms preprocess, 26.6ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 640)

0: 384x640 1 person, 1 car, 10.7ms
Speed: 4.1ms preprocess, 10.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 448x640 1 seatbelt, 12.7ms
Speed: 3.4ms preprocess, 12.7ms inference, 1.2ms p